# Imports

In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
import torchvision.transforms as T
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from skimage.color import gray2rgb

import numpy as np
from tqdm import tqdm
from train import *
from utils.data import LabelledSet, UnlabelledSet,FullRadiographDataset

In [2]:
ds = FullRadiographDataset("/datasets/pan-radiographs/", [1,2,3,4,5], None)

# Setup

In [2]:
epochs = 5
batch_size = 128
num_classes = 10

In [3]:
transforms = T.Compose([
    gray2rgb,
    T.ToTensor()
])
labelled, unlabelled = train_test_split(
    MNIST("/datasets", download=True, transform=transforms),
    test_size=.2,
    train_size=.8,
    shuffle=True
)

dataloader = DataLoader(
    labelled,
    batch_size=batch_size,
    shuffle=True,
)

criterion = nn.CrossEntropyLoss()

In [4]:
def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

model = torchvision.models.efficientnet_b0(weights="DEFAULT")
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)

opt = Adam(model.parameters(), lr=1e-3)

# Learn over annotated dataset

In [ ]:
train(
    model=model,
    epochs=5,
    criterion=criterion,
    dataloader=dataloader,
    opt=opt,
    metrics=[accuracy_score],
)

Training on NVIDIA GeForce GTX 1080 Ti


  0%|          | 0/375 [00:00<?, ?it/s]/home/david/Documentos/Pseudo-labelling/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
[Epoch 0]: 100%|██████████| 375/375 [00:15<00:00, 23.54it/s]


[Epoch 0] Loss: 0.21611 accuracy_score: 0.935 


[Epoch 1]: 100%|██████████| 375/375 [00:14<00:00, 25.17it/s]


[Epoch 1] Loss: 0.05825 accuracy_score: 0.984 


...

[Epoch 2] Loss: 0.04210 accuracy_score: 0.988 


[Epoch 3]: 100%|██████████| 375/375 [00:15<00:00, 24.99it/s]


[Epoch 3] Loss: 0.03497 accuracy_score: 0.989 


[Epoch 4]: 100%|██████████| 375/375 [00:15<00:00, 24.47it/s]

[Epoch 4] Loss: 0.02734 accuracy_score: 0.992 


In [15]:
test_loader = DataLoader(
    unlabelled,
    batch_size=batch_size,
    shuffle=True,
)

device = "cuda" if torch.cuda.is_available() else "cpu"

evaluate (
    model, 
    epochs,
    criterion,
    test_loader,
    [accuracy_score, lambda x,y: f1_score(x,y, average='micro')],
)

Testing on NVIDIA GeForce GTX 1080 Ti


Loss: 2.31453: 100%|██████████| 94/94 [00:01<00:00, 72.35it/s]

[Test Results] Loss: 2.31453 accuracy_score: 0.093 <lambda>: 0.093 


# Learn using pseudo-labels